## 범주형 변수 → 이항 변수화
범주형 변수 : male(0)/female(1), 10대(0)/20대(1)/.../80대(7), A/B/C/.../F<br>
연속형 변수와 범주형 변수를 잘 구분해야한다<br>
나이는 연속형 변수 X<br>
연속형 변수는 연산을 했을 때 의미있는 데이터가 생긴다<br>
범주형 변수 10대(0)에 +1을 하면 20대(1)이 된다 : 의미없는 연산<br><br>

### OneHotEncoder() 이항 변수화 함수
성별 : 남(0) → 10, 여(1) → 01<br>
연령 : 10대(0) → 10000000,20대(1) → 01000000, ... <br>


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [10]:
#[성별(여:0/남:1), 연령대(20:0/30:1/40:2), 성적(A:0/1/2/3/F:4)]
# 열 = feature, 차원
# 성별, 연령대, 성적 = 3개의 feature, 3차원
data = np.array([[0, 1, 1], # 여성, 30대, B학점
                 [0, 2, 2],
                 [1, 0, 3],
                 [1, 1, 4],
                 [0, 0, 0]])
data

array([[0, 1, 1],
       [0, 2, 2],
       [1, 0, 3],
       [1, 1, 4],
       [0, 0, 0]])

In [11]:
ohe = OneHotEncoder() # OneHotEncoder 객체 생성
ohe

OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True)

In [12]:
ohe.fit(data) # 어떤 데이터를 fit=one hot encoding시킬 것인가
# data변수에 저장된 데이터에 맞추어(fit) 이상변수화 시킴

C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True)

### 이항변수화가 잘 됐는지 알아보자 → 속성attribute이용

In [13]:
#[성별(여:0/남:1), 연령대(20:0/30:1/40:2), 성적(A:0/1/2/3/F:4)]
# 각 feature 10개를 나타냄
# [남, 여, 20대, 30대, 40대, A, B, C, D, F]
ohe.active_features_ 

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

In [16]:
# 각 feature 별 범주의 개수
ohe.n_values_

array([2, 3, 5])

In [18]:
# 각 feature의 index의 범위
ohe.feature_indices_
# 성별은 0이상 2미만, 연령대는 2이상 5미만, 성적은 5이상 10미만

array([ 0,  2,  5, 10], dtype=int32)

In [21]:
# 여성, 30대, D학점 [0, 1, 3]
# 0 → 10, 1 → 010, D → 00010
test = np.array([[0, 1, 3]])
ohe.transform(test).toarray() # OneHotEncoder().transform(2D array)
# 1X3 array → 1X10sparse matrix : 희소행렬
# array로 변환 후, ohe.feature_indices_을 사용해 끊어서 읽으면된다

array([[1., 0., 0., 1., 0., 0., 0., 0., 1., 0.]])

## 연속형 변수 → 이산화
np.digitize(mapping할 데이터, 구간) : 각 구간에 mapping <br>
np.digitize().np.where() : 연속형변수 → 이상화(2개 이상의 범주를 가지는 변수로 변환)

In [23]:
from pandas import DataFrame

In [25]:
np.random.seed(75)
df = DataFrame({
    "d1" : np.random.randn(10),
    "d2" : ["x", "x", "x", "x", "x", "y", "y", "y", "y", "y"]
})
df

,d1,d2
0,-0.709502,x
1,0.112694,x
2,0.477022,x
3,1.935981,x
4,0.450415,x
5,-1.188847,y
6,0.613631,y
7,-0.178142,y
8,1.346521,y
9,1.161150,y


In [38]:
print(df[["d1"]].min()) # dataframe
print(df["d1"].min()) # Series
print(df.d1.min()) # Series

d1   -1.188847
dtype: float64
-1.1888473584294919
-1.1888473584294919


In [37]:
bins = np.linspace(df.d1.min(), df.d1.max(), 5) # 범위 개수 
bins

array([-1.18884736, -0.40764024,  0.37356688,  1.154774  ,  1.93598112])

In [42]:
df["d1_bin"] = np.digitize(df["d1"], bins) # np.digitize(mapping할 데이터, 구간) : 각 구간에 mapping 
df # 5 = 최대값

,d1,d2,d1_bin
0,-0.709502,x,1
1,0.112694,x,2
2,0.477022,x,3
3,1.935981,x,5
4,0.450415,x,3
5,-1.188847,y,1
6,0.613631,y,3
7,-0.178142,y,2
8,1.346521,y,4
9,1.161150,y,4


## 데이터 그룹화

In [43]:
df.groupby("d1_bin") # pandas.groupby(group화의 기준이 되는 열) : group화된 객체 반환

In [45]:
df.groupby("d1_bin")["d1"].size() 
# [중요] "d1_bin"을 기준으로 group화를 한 뒤,
# d1 열을 참조하겠다

d1_bin
1    2
2    2
3    3
4    2
5    1
Name: d1, dtype: int64

다양한 함수를 그룹 단위로 적용가능

In [46]:
df.groupby("d1_bin")["d1"].mean()

d1_bin
1   -0.949175
2   -0.032724
3    0.513689
4    1.253836
5    1.935981
Name: d1, dtype: float64

In [47]:
df.groupby("d1_bin")["d1"].sum()

d1_bin
1   -1.898349
2   -0.065448
3    1.541068
4    2.507671
5    1.935981
Name: d1, dtype: float64

In [48]:
df.groupby("d1_bin")["d1"].std()

d1_bin
1    0.338948
2    0.205652
3    0.087568
4    0.131077
5         NaN
Name: d1, dtype: float64

In [49]:
df.groupby("d1_bin")["d2"].value_counts() 
# 그룹화된 각각의 그룹에 대해서 d2를 참조했을 때 각 value의 개수를 센다

d1_bin  d2
1       x     1
        y     1
2       x     1
        y     1
3       x     2
        y     1
4       y     2
5       x     1
Name: d2, dtype: int64

In [50]:
# 3번째 구간에 속하는 데이터만 추출
df["d1_bin"]

0    1
1    2
2    3
3    5
4    3
5    1
6    3
7    2
8    4
9    4
Name: d1_bin, dtype: int64

In [53]:
df2 = df[df["d1_bin"] == 3]
df2

,d1,d2,d1_bin
2,0.477022,x,3
4,0.450415,x,3
6,0.613631,y,3


np.where(조건식, 참, 거짓)
np.where(조건식, 참, 거짓, np.where(조건식, 참, 거짓))

In [54]:
df["d1"]

0   -0.709502
1    0.112694
2    0.477022
3    1.935981
4    0.450415
5   -1.188847
6    0.613631
7   -0.178142
8    1.346521
9    1.161150
Name: d1, dtype: float64

In [55]:
df["d1"].mean()

0.40209231875235396

In [57]:
df["d1"] >= df["d1"].mean()

0    False
1    False
2     True
3     True
4     True
5    False
6     True
7    False
8     True
9     True
Name: d1, dtype: bool

In [58]:
df["hl"] = np.where(df["d1"] >= df["d1"].mean(), "high", "row")
df

,d1,d2,d1_bin,hl
0,-0.709502,x,1,row
1,0.112694,x,2,row
2,0.477022,x,3,high
3,1.935981,x,5,high
4,0.450415,x,3,high
5,-1.188847,y,1,row
6,0.613631,y,3,high
7,-0.178142,y,2,row
8,1.346521,y,4,high
9,1.161150,y,4,high


In [59]:
df.groupby("hl")["d1"].size()

hl
high    6
row     4
Name: d1, dtype: int64

In [60]:
df.groupby("hl")["d1"].mean()

hl
high    0.997453
row    -0.490949
Name: d1, dtype: float64

In [61]:
df.groupby("hl")["d1"].std()

hl
high    0.591084
row     0.576501
Name: d1, dtype: float64

In [69]:
Q1 = np.percentile(df["d1"], 25) # d1열의 25%지점의 수
Q1

-0.10543282736762388

In [63]:
df["d1"]

0   -0.709502
1    0.112694
2    0.477022
3    1.935981
4    0.450415
5   -1.188847
6    0.613631
7   -0.178142
8    1.346521
9    1.161150
Name: d1, dtype: float64

In [70]:
Q3 = np.percentile(df["d1"], 75) # d1열의 75%지점의 수
Q3

1.0242701434033763

In [76]:
df["hml"] = np.where(df["d1"] >= Q3, "high", 
                     np.where(df["d1"] >= Q1, "medium", "low"))
df

,d1,d2,d1_bin,hl,hml
0,-0.709502,x,1,row,low
1,0.112694,x,2,row,medium
2,0.477022,x,3,high,medium
3,1.935981,x,5,high,high
4,0.450415,x,3,high,medium
5,-1.188847,y,1,row,low
6,0.613631,y,3,high,medium
7,-0.178142,y,2,row,low
8,1.346521,y,4,high,high
9,1.161150,y,4,high,high


In [82]:
data = np.arange(6).reshape(3,2)
data

array([[0, 1],
       [2, 3],
       [4, 5]])

In [ ]:
# data를 2차항 변수화
# x1, x2 → 1, x1, x2, x1^2, x1*x2, x2^2 
#         x1, x2
# array([[0, 1], → 1, 0, 1, 0, 0, 1
#       [2, 3], → 1, 2, 3, 4, 6, 9
#       [4, 5]]) → 1, 4, 5, 16, 20, 25

In [84]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures()
poly.fit_transform(data)

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

데이터 → 재구조화(피봇, 피봇테이블, 크로스테이블, melt, widetolog, stack/unstack) : 피봇테이블이 더 정교해서 피봇테이블만 알고있어도됨하지만 다른 사람들은 피봇도 쓸거니까 피봇도 알아둬


In [88]:
df = DataFrame({
    "c_id" : ["c1", "c1", "c1", "c2", "c2", "c2", "c3", "c3", "c3"],
    "p_id" : ["p1", "p2", "p3", "p1", "p2", "p3", "p1", "p2", "p3"],
    "amount" : [20, 10, 0, 30, 20, 30, 0, 5, 10]
})
df

,c_id,p_id,amount
0,c1,p1,20
1,c1,p2,10
2,c1,p3,0
3,c2,p1,30
4,c2,p2,20
5,c2,p3,30
6,c3,p1,0
7,c3,p2,5
8,c3,p3,10


In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
c_id      9 non-null object
p_id      9 non-null object
amount    9 non-null int64
dtypes: int64(1), object(2)
memory usage: 296.0+ bytes


### pivot()
pivot(row index, col index, 만나는 지점의 값) <br>
단수 재구조화 

In [93]:
df.pivot(index="c_id", columns="p_id", values="amount")

p_id,p1,p2,p3
c_id,,,
c1,20,10,0
c2,30,20,30
c3,0,5,10


### pivot_table()
pivot_table(index="c_id", columns="p_id", values="amount")

In [94]:
df.pivot_table(index="c_id", columns="p_id", values="amount")

p_id,p1,p2,p3
c_id,,,
c1,20,10,0
c2,30,20,30
c3,0,5,10


### pivot과 pivot_table()의 차이
* 둘 이상의 변수로 index를 삼고자하는 경우 <br>
error O : pivot(index=["sido", "dong"], columns="p_id", values="amount") <br>
error X : pivot_table(index=["sido", "dong"], columns="p_id", values="amount") <br><br>

* 집계함수 가능 여부
그룹단위 sum, mean 등 <br>
pivot()(사용 불가), pivot_table()(사용 가능) <br>
pivot_table(index=["sido", "dong"], columns="p_id", values="amount", aggfunc=np.sum)


In [96]:
df = DataFrame({
    "c_id" : ["c1", "c1", "c1", "c2", "c2", "c2", "c3", "c3", "c3"],
    "p_id" : ["p1", "p2", "p3", "p1", "p2", "p3", "p1", "p2", "p3"],
    "amount" : [20, 10, 0, 30, 20, 30, 0, 5, 10],
    "reg" : ["S", "S", "S", "S", "S", "S", "A", "A", "A"] 
})
df

,c_id,p_id,amount,reg
0,c1,p1,20,S
1,c1,p2,10,S
2,c1,p3,0,S
3,c2,p1,30,S
4,c2,p2,20,S
5,c2,p3,30,S
6,c3,p1,0,A
7,c3,p2,5,A
8,c3,p3,10,A


In [99]:
df.pivot(index=["c_id, "reg"], columns="p_id", values="amount")

SyntaxError: invalid syntax (<ipython-input-99-225c8f23e125>, line 1)

In [100]:
df.pivot_table(index=["c_id", "reg"], columns="p_id", values="amount")

,p_id,p1,p2,p3
c_id,reg,,,
c1,S,20,10,0
c2,S,30,20,30
c3,A,0,5,10


In [102]:
df.pivot_table(index=["reg"], columns="p_id", values="amount") # default :  aggfunc=np.mean
df.pivot_table(index=["reg"], columns="p_id", values="amount", aggfunc=np.mean)

p_id,p1,p2,p3
reg,,,
A,0,5,10
S,25,15,15


In [103]:
df.pivot_table(index=["reg"], columns="p_id", values="amount", aggfunc=np.sum)

p_id,p1,p2,p3
reg,,,
A,0,5,10
S,50,30,30


In [106]:
df1 = pd.read_csv("./res/concat_1.csv")
df2 = pd.read_csv("./res/concat_2.csv")
df3 = pd.read_csv("./res/concat_3.csv")

In [109]:
row_concat = pd.concat([df1, df2, df3]) # df1, df2, df3 병합
row_concat

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3
0,a4,b4,c4,d4
1,a5,b5,c5,d5
2,a6,b6,c6,d6
3,a7,b7,c7,d7
0,a8,b8,c8,d8
1,a9,b9,c9,d9


### 특정행 데이터 추출
### ix
integer position과 label모두 사용 할 수 있다. <br>
만약 label이 숫자라면 label-based index만 된다. <br><br>

#### iloc
iloc = integerloc <br>
integer positon를 통해 값을 찾을 수 있다. label로는 찾을 수 없다. <br>
눈에는 안보이지만 순서대로 indexing되어져있음

In [114]:
# row_concat의 4번째 행 추출 → index가 3
row_concat.iloc[3, ]

A    a3
B    b3
C    c3
D    d3
Name: 3, dtype: object

In [115]:
row_concat.iloc[4, ]

A    a4
B    b4
C    c4
D    d4
Name: 0, dtype: object

In [118]:
# df1에 series를 연결
new_row = pd.Series(["n1", "n2", "n3", "n4"])
df1 

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3


In [119]:
pd.concat([df1, new_row])

,A,B,C,D,0
0,a0,b0,c0,d0,NaN
1,a1,b1,c1,d1,NaN
2,a2,b2,c2,d2,NaN
3,a3,b3,c3,d3,NaN
0,NaN,NaN,NaN,NaN,n1
1,NaN,NaN,NaN,NaN,n2
2,NaN,NaN,NaN,NaN,n3
3,NaN,NaN,NaN,NaN,n4


In [120]:
new_row = pd.DataFrame([["n1", "n2", "n3", "n4"]], columns=["A", "B", "C", "D"])

In [121]:
pd.concat([df1, new_row])

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3
0,n1,n2,n3,n4


pandas.Series는 col name이 없기 때문에, df의 row로 연결할 수 없었다 <br>
pandas.Series → pandas.Dataframe로 변환 후, 연결

In [122]:
# append 사용 연결 가능
# concat은 2개 이상의 DF를 연결
# append는 1개의 DF만 연결 가능
df1.append(new_row)

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3
0,n1,n2,n3,n4


In [124]:
data_dict = {"A" : "n1", "B" : "n2", "C" : "n3", "D" : "n4"}
df1.append(data_dict)

TypeError: Can only append a Series if ignore_index=True or if the Series has a name

In [125]:
df1.append(data_dict, ignore_index=True)

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3
4,n1,n2,n3,n4


In [129]:
col_concat = pd.concat([df1, df2, df3], axis=1) # 좌우로 병합
col_concat

,A,B,C,D,A,B,C,D,A,B,C,D
0,a0,b0,c0,d0,a4,b4,c4,d4,a8,b8,c8,d8
1,a1,b1,c1,d1,a5,b5,c5,d5,a9,b9,c9,d9
2,a2,b2,c2,d2,a6,b6,c6,d6,a10,b10,c10,d10
3,a3,b3,c3,d3,a7,b7,c7,d7,a11,b11,c11,d11


In [132]:
col_concat["A"]

,A,A,A
0,a0,a4,a8
1,a1,a5,a9
2,a2,a6,a10
3,a3,a7,a11
